In [1]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:

preprocessed_credit_risk_df = pd.read_csv('preprocessed_credit_risk.csv')

preprocessed_credit_risk_df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
0,22,59000,50.0,35000,16.02,1,0.59,3,False,False,True,False,False,True,False,False,False,True
1,24,83000,8.0,35000,8.90,1,0.42,2,False,False,True,False,False,False,False,False,False,False
2,25,137000,9.0,34800,16.77,0,0.25,2,False,False,True,False,False,False,True,False,False,True
3,24,10980,0.0,1500,7.29,0,0.14,3,False,True,False,False,False,False,False,False,False,False
4,22,80000,3.0,33950,14.54,1,0.42,4,False,False,True,False,False,True,False,False,False,True
